# How to choose the Trotter steps for an MPF

This guide teaches you how to find a good set of Trotter steps for an MPF.
We assume that you have already read and understood the [Getting started](https://qiskit.github.io/qiskit-addon-mpf/tutorials/01_getting_started.html) tutorial.

## Understanding some heuristics

In principle, any $k_j$ values can be chosen but some choices of Trotter steps will lead to a larger noise amplification on real devices than others.
Thus, it is important that one tries to find _"good"_ values of $k_j$.
In the following we will discuss some heuristics to help guide you in doing so.

1. In practice, our **largest** $k_j$ ($k_{\text{max}}$) value is simply bound by the depth of quantum circuit we can reliably execute.

2. Since the Trotter error is only well behaved for $\text{d}t = t/k \lt 1$ (see [Understanding the stability of MPFs](https://qiskit.github.io/qiskit-addon-mpf/explanations/mpf_stability.html)), our **smallest** $k_j$ ($k_{\text{min}}$) should satisfy: $t/k_{\text{min}} \lt 1$ (Note: empirically $\text{d}t \leq 1$ seems to work fine, too).

3. None of our $x_j$ values should be close to $0$, as this would imply that we might as well exclude the corresponding $k_j$ from the expansion.

4. Similarly, we do not want the $x_j$ for $k_{\text{max}}$ to be the only dominant one, since that implies that we are essentially just recovering a single product formula with $k=k_{\text{max}}$.

5. Finally, we want the L1-norm of our coefficients, $x_j$, to be small, as this indicates a well-conditioned MPF (see [Carrera Vazquez et al., 2023]). Furthermore, small coefficients minimize the variance amplification of the MPF.

[Carrera Vazquez et al., 2023]: https://quantum-journal.org/papers/q-2023-07-25-1067/

## Applying these heuristics

Reusing the example from the [Getting started](https://qiskit.github.io/qiskit-addon-mpf/tutorials/01_getting_started.html) tutorial, we want to simulate up to time $t=8.0$.
Thus, $k_{\text{min}}=8$.

For the sake of this example, let us assume that our deepest circuit is bound by $k_{\text{max}}=20$.

In addition to the constraints above, we will follow the heuristic presented by [Zhuk et al., 2023], where they minimize a re-weighted L1-norm of the coefficients:

$$
\text{min}_{k_j} \sum_j \frac{|c_j|}{k_j^{2\chi}} \, ,
$$

where $\chi$ is the order of our individual product formulas.

[Zhuk et al., 2023]: https://arxiv.org/abs/2306.12569

### Preparing our LSE

In a first step, we prepare the LSE, $Ax=b$, leveraging a feature of the [setup_lse](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.setup_lse.html) function that allows us to use a [cvxpy.Parameter](https://www.cvxpy.org/api_reference/cvxpy.expressions.html#parameter) object for our $k_j$ values.

In [30]:
order = 2
symmetric = True

min_k = 8
max_k = 20

max_l1_norm = 5.0
min_coeff = 0.01

In [31]:
import cvxpy as cp
from qiskit_addon_mpf.static import setup_lse

ks = cp.Parameter(3, integer=True)
lse = setup_lse(ks, order=order, symmetric=symmetric)

Next, we loop over all possible 3-tuples of $k_j$ values within our specified range.
For each one, we compute the analytical coefficients, $x_j$, and then apply our filter criteria.
We keep track of those $k_j$ tuples which satisfy our criteria.

In [32]:
import numpy as np

possible_ks = {}

for k1 in range(min_k, max_k):
    for k2 in range(k1 + 1, max_k):
        for k3 in range(k2 + 1, max_k):
            ks.value = [k1, k2, k3]
            coeffs = lse.solve()

            if np.any(np.isclose(coeffs, 0.0, atol=min_coeff)):
                continue

            norm1 = np.linalg.norm(coeffs, ord=1)
            if norm1 > max_l1_norm:
                continue

            weighted = np.sum([np.abs(c) / k ** (2 * order) for c, k in zip(coeffs, ks.value)])
            possible_ks[tuple(int(k) for k in ks.value)] = {
                "coeffs": tuple(coeffs),
                "weighted": float(weighted),
                "norm1": float(norm1),
            }

Finally, we sort our $k_j$ by the re-weighted L1-norm presented by [Zhuk et al., 2023]:

[Zhuk et al., 2023]: https://arxiv.org/abs/2306.12569

In [33]:
for ks_val in sorted(possible_ks, key=lambda ks_val: possible_ks[ks_val].get("weighted")):
    print(ks_val, possible_ks[ks_val])

(8, 14, 19) {'coeffs': (0.10447913478216586, -1.7638200183654775, 2.6593408835833117), 'weighted': 9.182736455463762e-05, 'norm1': 4.527640036730955}
(8, 13, 19) {'coeffs': (0.13134519801186473, -1.4167162698412699, 2.285371071829405), 'weighted': 9.920634920634922e-05, 'norm1': 3.8334325396825397}
(8, 12, 19) {'coeffs': (0.17239057239057254, -1.1944700460829496, 2.022079473692377), 'weighted': 0.00011520737327188946, 'norm1': 3.388940092165899}
(9, 13, 19) {'coeffs': (0.2662743506493499, -1.6904000946969673, 2.4241257440476174), 'weighted': 0.00011837121212121191, 'norm1': 4.380800189393934}
(8, 13, 18) {'coeffs': (0.15003663003663004, -1.7549001536098316, 2.6048635235732016), 'weighted': 0.00012288786482334872, 'norm1': 4.509800307219663}
(8, 12, 18) {'coeffs': (0.19692307692307653, -1.4399999999999984, 2.243076923076922), 'weighted': 0.0001388888888888887, 'norm1': 3.879999999999997}
(8, 11, 19) {'coeffs': (0.24195168054817162, -1.070248538011695, 1.8282968574635234), 'weighted': 0.

Recalling the [Getting started](https://qiskit.github.io/qiskit-addon-mpf/tutorials/01_getting_started.html) tutorial, from these possible values we picked `ks=(8, 12, 19)` since it strikes a good balance of minimizing the re-weighted and plain L1-norms.